In [1]:
import netaddr, copy
import pandas as pd
from pandas import Series
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.compose import make_column_transformer

In [2]:
train_users = pd.read_csv("data/train_users.csv")
train_logs = pd.read_csv("data/train_logs.csv")
train_y = pd.read_csv("data/train_y.csv")
ips = pd.read_csv("data/ip2location.csv")
#model.fit(train_users, train_logs, train_y)

In [10]:
class UserPredictor():
    def fit(self, users, logs, ys):
        self.users = self.total_min_ips(users, logs)
        self.users = self.ip_to_country(self.users)
        self.merged = pd.merge(left=self.users, right=ys, left_on="id", right_on="id")
        self.pipeline = Pipeline([
    ("both", make_column_transformer((OneHotEncoder(), ["badge", "age", "country"]), 
                                     (PolynomialFeatures(degree=1, include_bias=False), ["past_purchase_amt", "total_min", "avg_minutes", "num_visited"]), remainder="passthrough")),
    ("lr", LogisticRegression(max_iter=1025))
])
        self.pipeline.fit(self.merged[["badge", "age", "country", "past_purchase_amt", "total_min", "avg_minutes", "num_visited"]], self.merged["y"])
    
    def predict(self, users, logs):
        self.users = self.total_min_ips(users, logs)
        self.users = self.ip_to_country(self.users)
        self.users["predicted"] = self.pipeline.predict(self.users[["badge", "age", "country", "past_purchase_amt", "total_min", "avg_minutes", "num_visited"]])
        return self.users["predicted"].values
    
    def total_min_ips(self, users, logs):    
        minutes = {}
        avg_min = {}
        num_visited = {}
        netaddrs = {}
        ids = set(users["id"])
        for id_ in ids:
            minutes[id_] = logs[logs["id"] == id_]["minutes_on_page"].sum()
            avg_min[id_] = logs[logs["id"] == id_]["minutes_on_page"].mean()
            num_visited[id_] = len(logs[logs["id"] == id_])
            try:
                netaddrs[id_] = int(netaddr.IPAddress(logs[logs["id"] == id_]["ip_address"].values[-1]))
            except:
                netaddrs[id_] = 0
        users["total_min"] = minutes.values()
        users["avg_minutes"] = avg_min.values()
        users["num_visited"] = num_visited.values()
        users["netaddr"] = netaddrs.values()
        return users.fillna(0)
    
    def ip_to_country(self, df):
        df = df.sort_values("netaddr")
        countries = list()
        ip_idx = 0
        for ip in df["netaddr"]:
            for low in ips["low"][ip_idx:]:
                if low <= ip <= ips["high"][ip_idx]:
                    countries.append(ips["region"][ip_idx])
                    break
                else:
                    ip_idx += 1
        df["country"] = countries
        return df.sort_values("id")

In [11]:
model = UserPredictor()

In [5]:
model.fit(train_users, train_logs, train_y)

/home/seanwells/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [6]:
test_users = pd.read_csv("data/test1_users.csv")
test_logs = pd.read_csv("data/test1_logs.csv")

In [7]:
y_pred = model.predict(test_users, test_logs)

In [9]:
len(y_pred)

20000